In [ ]:
from battery_analysis import get_engine
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.engine import Engine

In [ ]:
engine, schema = get_engine(schema="diffusion_results_policy_20250831_020659713296")
year = 2026
tbl = "agent_annual_finance"

sql = f"""
        SELECT * FROM {schema}.{tbl}
        WHERE year = :year
        ORDER BY agent_id
        """
params = {"year": year}

finance = pd.read_sql(text(sql), engine, params=params)

tbl = "agent_annual_finance_detail"

sql = f"""
        SELECT * FROM {schema}.{tbl}
        WHERE year = :year
        ORDER BY agent_id, year, scenario_case, metric, period_index
        """
params = {"year": year}

finance_det = pd.read_sql(text(sql), engine, params=params)

In [ ]:
# Get energy value
energy_value = finance_det[
    (finance_det['period_index']==1) &
    (finance_det['metric'].isin(['cf_energy_value', 'cf_debt_payment_total']))
     ][['agent_id', 'value', 'scenario_case', 'metric']]
energy_value_wide = energy_value.pivot(index=['agent_id', 'scenario_case'], columns='metric', values='value').reset_index()

# Get total upfront cost
upfront_cost = finance_det[
    (finance_det['metric'] == 'cf_discounted_costs')
    ].groupby(['agent_id', 'scenario_case'], as_index = False).agg(upfront_cost=("value","sum"))

# Merge
upfront_cost_value = upfront_cost.merge(energy_value_wide, on = ['agent_id', 'scenario_case'], how="left")

# Merge NPV and system sizing
upfront_cost_value_size = upfront_cost_value.merge(finance[['agent_id', 'scenario_case', 'system_kw', 
                                                            'batt_kw', 'batt_kwh', 'npv_usd', 'payback_yrs']],
                                                            on = ['agent_id', 'scenario_case'])

upfront_cost_value_size['upfront_cost']          = upfront_cost_value_size['upfront_cost'].apply(lambda x: str(int(round(x,0))) if pd.notna(x) else x)
upfront_cost_value_size['cf_energy_value']       = upfront_cost_value_size['cf_energy_value'].apply(lambda x: str(int(round(x,0))) if pd.notna(x) else x)
upfront_cost_value_size['cf_debt_payment_total'] = upfront_cost_value_size['cf_debt_payment_total'].apply(lambda x: str(int(round(x,0))) if pd.notna(x) else x)
upfront_cost_value_size['npv_usd']               = upfront_cost_value_size['npv_usd'].apply(lambda x: str(int(round(x,0))) if pd.notna(x) else x)
upfront_cost_value_size['system_kw']   = upfront_cost_value_size['system_kw'].apply(lambda x: f"{x:.2f}".rstrip("0").rstrip(".") if pd.notna(x) else x)

upfront_cost_value_size['batt_kw']   = upfront_cost_value_size['batt_kw'].apply(lambda x: f"{x:.2f}".rstrip("0").rstrip(".") if pd.notna(x) else x)
upfront_cost_value_size['batt_kwh']  = upfront_cost_value_size['batt_kwh'].apply(lambda x: f"{x:.2f}".rstrip("0").rstrip(".") if pd.notna(x) else x)
upfront_cost_value_size['payback_yrs'] = upfront_cost_value_size['payback_yrs'].apply(lambda x: f"{x:.2f}".rstrip("0").rstrip(".") if pd.notna(x) else x)



df_wide = (
    upfront_cost_value_size
    .melt(id_vars=['agent_id', 'scenario_case'], 
          value_vars = [col for col in upfront_cost_value_size.columns if col not in ['agent_id', 'scenario_case']],
          var_name='metric')
    .pivot(index = ["agent_id", "metric"], columns=["scenario_case"], 
           values = ['value'])
    .reset_index()
    )

#df_wide['diff'] = df_wide['value']['pv_batt'] - df_wide['value']['pv_only']



In [ ]:
df_wide[df_wide['agent_id']==483916]

In [ ]:
hourly[hourly['variable'] == 'grid_to_load_util_kwh']

In [ ]:
engine, schema = get_engine(schema="diffusion_results_policy_20250831_020659713296")
year = 2026
tbl = "agent_hourly_econ"

sql = f"""
        SELECT * FROM {schema}.{tbl}
        WHERE agent_id = 483916
        ORDER BY hour_index
        """

hourly = pd.read_sql(text(sql), engine, params=params)
